In [ ]:
import pandas as pd

In [ ]:
#remove NaN for better visibility
users_df = pd.read_csv('users.csv').fillna('')
repositories_df = pd.read_csv('repositories.csv').fillna('')

In [ ]:
users_df.head(2) # view first 2 records to verify

In [ ]:
# no of hireable users with email
hireable_with_email = users_df[(users_df['hireable'] == True) & (users_df['email']!='')].shape[0]
#no of non hireable users with email
non_hireable_with_email = users_df[(users_df['hireable'] != True) & (users_df['email']!='')].shape[0]

# total count of hireable
total_hireable = users_df[users_df['hireable'] == True].shape[0]
# total count of non hireable
total_non_hireable = users_df[users_df['hireable'] != True].shape[0]

# Calculate the fractions
fraction_hireable_with_email = hireable_with_email / total_hireable if total_hireable > 0 else 0
fraction_non_hireable_with_email = non_hireable_with_email / total_non_hireable if total_non_hireable > 0 else 0

# Calculate the difference
difference = fraction_hireable_with_email - fraction_non_hireable_with_email

# Display the result
print("Difference in fractions of users with email addresses:", difference)


In [ ]:
#get company where most work
users_df[users_df['company'].notna()]['company'].value_counts().nlargest(1)

In [ ]:
# Get top 3 non-empty license name with  counts
top_3_licenses = repositories_df[repositories_df['language'] != '']['language'].value_counts().nlargest(3)
top_3_licenses

,count
language,
JavaScript,5464
Python,2797
TypeScript,1457


In [ ]:
#first filter who are users who made their id after 2020
users_after_2020 = users_df[pd.to_datetime(users_df['created_at']) > '2020-01-01']

# get the login id of these in a list
logins_after_2020 = users_after_2020['login'].tolist()

# filter by checking if users are in list
repos_after_2020 = repositories_df[repositories_df['login'].isin(logins_after_2020)]

# remove empty language entries
non_empty_languages = repos_after_2020[repos_after_2020['language'] != '']

language_counts = non_empty_languages['language'].value_counts()

# get top 2 most popular, need only second though
second_most_popular_language = language_counts.nlargest(2)
second_most_popular_language

,count
language,
Python,112
TypeScript,22


In [ ]:
# 5 earliest users
five_earliest_users = users_df.nsmallest(5, 'created_at')
five_earliest_users

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
277,dylanegan,Dylan Egan,,"Sydney, Australia",,,,55,126,0,2008-01-29 23:15:18+00:00
235,cjheath,Clifford Heath,,Sydney Australia,clifford.heath@gmail.com,,"Enterprise software product architect, data mo...",85,155,19,2008-02-29 06:55:16+00:00
362,freshtonic,James Sadler,,"Sydney, Australia",freshtonic@gmail.com,,,60,103,94,2008-03-10 21:01:27+00:00
368,dhowden,David Howden,ALGOLIA (PREV @SAJARI),"Sydney, Australia",dhowden@gmail.com,,,41,101,32,2008-03-11 15:58:04+00:00
121,mikel,Mikel Lindsaar,"STORECONNECT, METAPULSE, REINTERACTIVE","Sydney, Australia",,True,,34,277,0,2008-03-18 14:05:00+00:00


In [ ]:
# make column of leader strength
users_df['leader_strength'] = (users_df['followers']/(1+ users_df['following']))
users_df.get(users_df['leader_strength'].max())

In [ ]:
# get correlation bw projects and wikis
new_rep_df = pd.DataFrame()
new_rep_df['has_projects'] = repositories_df['has_projects'].apply(lambda x: 1 if str(x).lower() == 'true' else 0)
new_rep_df['has_wiki'] = repositories_df['has_wiki'].apply(lambda x: 1 if str(x).lower() == 'true' else 0)
new_rep_df.corr()

,has_projects,has_wiki
has_projects,1.000000,0.251262
has_wiki,0.251262,1.000000


In [ ]:
# Calculate average following for hireable and non-hireable
avg_following_hireable = users_df[users_df['hireable'] == True]['following'].mean()
avg_following_non_hireable = users_df[users_df['hireable'] == '']['following'].mean()

difference = avg_following_hireable - avg_following_non_hireable

print("Difference in average following:", difference)

Difference in average following: 52.655219083065475


In [ ]:
#split them by space and get the last part
users_df['surname'] = users_df['name'].str.split(' ').str[-1]
users_df['surname'].value_counts().head()

,count
surname,
,10
Zhang,4
Wu,4
Huang,3
Nguyen,3


In [ ]:
import statsmodels.api as sm
# Filter out users without bios
filtered_users_df = users_df[(users_df['bio'] != '')]

# Split by space
filtered_users_df['bio_length'] = filtered_users_df['bio'].apply(lambda x: len(x.split()))

# get bio length and followers in variables
X = filtered_users_df[['bio_length']]  # Independent variable
y = filtered_users_df['followers']     # Dependent variable

# Step 4: Add a constant to the independent variable (required for statsmodels)
X = sm.add_constant(X)

# Step 5: Fit the regression model
model = sm.OLS(y, X).fit()

# Step 6: Extract and round the slope (coefficient for bio_length)
slope = model.params['bio_length']
slope_rounded = slope

# Display the result
print("Regression slope of followers on bio word count:", slope_rounded)

Regression slope of followers on bio word count: -11.011081993736079


<ipython-input-39-967220301c31>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_users_df['bio_length'] = filtered_users_df['bio'].apply(lambda x: len(x.split()))
